In [131]:
table = r"""
% Please add the following required packages to your document preamble:
% \usepackage{multirow}
\begin{table}[]
\caption{caption here}
\label{tab:my-table}
\begin{tabular}{llrrlllllll}
\hline
\multicolumn{5}{l}{\multirow{2}{*}{}} & \multicolumn{2}{c}{\multirow{2}{*}{CityScapes}} & \multicolumn{2}{l}{\multirow{2}{*}{}} & \multicolumn{2}{c}{\multirow{2}{*}{ACDC}} \\
\multicolumn{5}{l}{} & \multicolumn{2}{c}{} & \multicolumn{2}{l}{} & \multicolumn{2}{c}{} \\ \cline{6-7} \cline{10-11} 
 &  & \multicolumn{1}{l}{$\sigma$} & \multicolumn{1}{l}{R} &  & info. gain & $\%\oslash$ &  &  & info. gain & $\%\oslash$ \\ \hline
\multirow{2}{*}{} & \multirow{2}{*}{Uncertified HrNet} & \multicolumn{1}{l}{\multirow{2}{*}{-}} & \multicolumn{1}{l}{\multirow{2}{*}{-}} & \multirow{2}{*}{} & \multirow{2}{*}{valuehere1} & \multirow{2}{*}{valuehere14} & \multirow{2}{*}{} & \multirow{2}{*}{} & \multirow{2}{*}{valuehere27} & \multirow{2}{*}{valuehere40} \\
 &  & \multicolumn{1}{l}{} & \multicolumn{1}{l}{} &  &  &  &  &  &  &  \\
\multirow{7}{*}{$n = 100$} & \multirow{3}{*}{\begin{tabular}[c]{@{}l@{}}SegCertify\\ $\tau=0.75$\end{tabular}} & 0.25 & 0.17 &  & valuehere2 & valuehere15 &  &  & valuehere28 & valuehere41 \\
 &  & 0.33 & 0.22 &  & valuehere3 & valuehere16 &  &  & valuehere29 & valuehere42 \\
 &  & 0.5 & 0.34 &  & valuehere4 & valuehere17 &  &  & valuehere30 & valuehere43 \\
 & \multicolumn{10}{l}{} \\
 & \multirow{3}{*}{\begin{tabular}[c]{@{}l@{}}AdaptiveCertify\\ $\tau=0.75$\end{tabular}} & 0.25 & 0.17 &  & valuehere5 & valuehere18 &  &  & valuehere31 & valuehere44 \\
 &  & 0.33 & 0.22 &  & valuehere6 & valuehere19 &  &  & valuehere32 & valuehere45 \\
 &  & 0.5 & 0.34 &  & valuehere7 & valuehere20 &  &  & valuehere33 & valuehere46 \\ \hline
\multirow{7}{*}{$n=500$} & \multirow{3}{*}{\begin{tabular}[c]{@{}l@{}}SegCertify\\ $\tau=0.95$\end{tabular}} & 0.25 & 0.41 &  & valuehere8 & valuehere21 &  &  & valuehere34 & valuehere47 \\
 &  & 0.33 & 0.52 &  & valuehere9 & valuehere22 &  &  & valuehere35 & valuehere48 \\
 &  & 0.5 & 0.82 &  & valuehere10 & valuehere23 &  &  & valuehere36 & valuehere49 \\
 & \multicolumn{10}{l}{} \\
 & \multirow{3}{*}{\begin{tabular}[c]{@{}l@{}}AdaptiveCertify\\ $\tau=0.95$\end{tabular}} & 0.25 & 0.41 &  & valuehere11 & valuehere24 &  &  & valuehere37 & valuehere50 \\
 &  & 0.33 & 0.52 &  & valuehere12 & valuehere25 &  &  & valuehere38 & valuehere51 \\
 &  & 0.5 & 0.82 &  & valuehere13 & valuehere26 &  &  & valuehere39 & valuehere52 \\ \hline
\end{tabular}
\end{table}
""" 

In [132]:
import pickle
import os
from os.path import dirname, abspath
import glob
import numpy as np
from plotly.subplots import make_subplots

os.chdir('/BS/mlcysec2/work/adaptive-hierarchical-certification/')

experiment_name = 'table'
dicts = glob.glob(f'cityscapes_logs/slurm_logs/{experiment_name}/frankfurt_000000_020321.pkl')

overall_d = dict()
for im_dict in dicts:
    d = pickle.load(open(im_dict, 'rb'))
    for im_name_k in d.keys():
        overall_d[im_name_k] = d[im_name_k]
graph_dict = {} # should have (n, n0, th_func, i) as keys, values: sum
for name, name_dict in overall_d.items():
    for k in name_dict.keys():
        if k not in graph_dict:
            graph_dict[k] = {'num_pixels':0, 
                            'certified_pixels':0, 
                            'uncertified_pos':0,
                            'c_info_gain':0, 
                            'certified_pos':0, 
                            'boundary_pixels':0, 
                            'nonboundary_pixels':0, 
                            'pos_certified_boundary_pixels':0, 
                            'pos_certified_nonboundary_pixels':0,
                            'num_h_pixels':{}, 'pos_h_pixels':{},
                            'ig_per_class_dict': {i:{'ig': np.zeros(19), 'pixels_count': np.zeros(19)} for i in range(4)}}
        if k == (None, None, None, 0, None, None):
            graph_dict[k]['uncertified_pos'] += name_dict[k]['uncertified_pos']
            print('base')
            continue
        graph_dict[k]['num_pixels'] += name_dict[k]['num_pixels']
        graph_dict[k]['certified_pixels'] += name_dict[k]['certified_pixels']
        graph_dict[k]['certified_pos'] += name_dict[k]['certified_pos']
        graph_dict[k]['c_info_gain'] += name_dict[k]['c_info_gain']
        for i in name_dict[k]['ig_per_class_dict'].keys():
            graph_dict[k]['ig_per_class_dict'][i]['ig'] += name_dict[k]['ig_per_class_dict'][i]['ig']
            graph_dict[k]['ig_per_class_dict'][i]['pixels_count'] += name_dict[k]['ig_per_class_dict'][i]['pixels_count']

base


In [129]:
k = (500, 10, '(0, 0, 0.25)', 4, 0.5, 0.95)

sub_k = ['uncertified_pos'] + ['c_info_gain']*12 + ['certified_pixels']*12
num_pixels = graph_dict[k]['num_pixels']
t_d = {
    # cityscapes info gain
    'valuehere1': (None, None, None, 0, None, None),
    'valuehere2': (100, 10, None, 0, 0.25, 0.75),
    'valuehere3': (100, 10, None, 0, 0.33, 0.75),
    'valuehere4': (100, 10, None, 0, 0.5, 0.75),

    'valuehere5': (100, 10, '(0, 0, 0.25)', 4, 0.25, 0.75),
    'valuehere6': (100, 10, '(0, 0, 0.25)', 4, 0.33, 0.75),
    'valuehere7': (100, 10, '(0, 0, 0.25)', 4, 0.5, 0.75),
    
    'valuehere8': (500, 10, None, 0, 0.25, 0.95),
    'valuehere9': (500, 10, None, 0, 0.33, 0.95),
    'valuehere10': (500, 10, None, 0, 0.5, 0.95),

    'valuehere11': (500, 10, '(0, 0, 0.25)', 4, 0.25, 0.95),
    'valuehere12': (500, 10, '(0, 0, 0.25)', 4, 0.33, 0.95),
    'valuehere13': (500, 10, '(0, 0, 0.25)', 4, 0.5, 0.95),
    # start of abstain
    'valuehere15': (100, 10, None, 0, 0.25, 0.75),
    'valuehere16': (100, 10, None, 0, 0.33, 0.75),
    'valuehere17': (100, 10, None, 0, 0.5, 0.75),

    'valuehere18': (100, 10, '(0, 0, 0.25)', 4, 0.25, 0.75),
    'valuehere19': (100, 10, '(0, 0, 0.25)', 4, 0.33, 0.75),
    'valuehere20': (100, 10, '(0, 0, 0.25)', 4, 0.5, 0.75),
    
    'valuehere21': (500, 10, None, 0, 0.25, 0.95),
    'valuehere22': (500, 10, None, 0, 0.33, 0.95),
    'valuehere23': (500, 10, None, 0, 0.5, 0.95),

    'valuehere24': (500, 10, '(0, 0, 0.25)', 4, 0.25, 0.95),
    'valuehere25': (500, 10, '(0, 0, 0.25)', 4, 0.33, 0.95),
    'valuehere26': (500, 10, '(0, 0, 0.25)', 4, 0.5, 0.95),

    }
for str_to_replace, model_k, sub_k in zip(t_d.keys(), t_d.values(), sub_k):
    val = graph_dict[model_k][sub_k]
    if sub_k == 'c_info_gain': val = str(round(val/num_pixels/np.log(19), 2))
    elif sub_k == 'certified_pixels': val = str(round((num_pixels - val)/num_pixels, 2))
    elif sub_k == 'uncertified_pos':  val = str(round(val/num_pixels, 2)); print('here')
    table = table.replace(f'{str_to_replace} &', f'${val}$ &')
    table = table.replace(str_to_replace+'}', '$'+val+'$}')

here


In [130]:
text_file = open(f"cityscapes_logs/slurm_logs/{experiment_name}/table.txt", "w")
 
#write string to file
text_file.write(table)
 
#close file
text_file.close()

In [31]:
import json
json.dump({'table': table}, open(f'cityscapes_logs/slurm_logs/{experiment_name}/table.txt', 'w'))